In [10]:
from line_solver import *
import numpy as np
import time
GlobalConstants.set_verbose(VerboseLevel.STD)

In [11]:
# Create network
model = Network('model')

# Block 1: nodes (all queues, no delay)
node1 = Queue(model, 'Queue1', SchedStrategy.PS)
node2 = Queue(model, 'Queue2', SchedStrategy.PS)
node3 = Queue(model, 'Queue3', SchedStrategy.PS)
node3.set_number_of_servers(3)  # Queue3 has 3 servers

In [12]:
# Block 2: classes - each with 1 job
N = [1, 1, 1, 1]  # Population for each class
jobclass1 = ClosedClass(model, 'Class1', N[0], node1, 0)
jobclass2 = ClosedClass(model, 'Class2', N[1], node1, 0)
jobclass3 = ClosedClass(model, 'Class3', N[2], node1, 0)
jobclass4 = ClosedClass(model, 'Class4', N[3], node1, 0)

In [13]:
# Set service times for Queue1
node1.set_service(jobclass1, Exp.fit_mean(1.0))
node1.set_service(jobclass2, Exp.fit_mean(1.0/2.0))
node1.set_service(jobclass3, Exp.fit_mean(1.0))
node1.set_service(jobclass4, Exp.fit_mean(1.0))

# Set service times for Queue2
node2.set_service(jobclass1, Exp.fit_mean(1.0/3.0))
node2.set_service(jobclass2, Exp.fit_mean(1.0/4.0))
node2.set_service(jobclass3, Exp.fit_mean(1.0/5.0))
node2.set_service(jobclass4, Exp.fit_mean(1.0))

# Set service times for Queue3
node3.set_service(jobclass1, Exp.fit_mean(1.0))
node3.set_service(jobclass2, Exp.fit_mean(1.0/3.0))
node3.set_service(jobclass3, Exp.fit_mean(1.0/5.0))
node3.set_service(jobclass4, Exp.fit_mean(1.0/2.0))

In [14]:
# Block 3: routing with class switching
K = 4  # Number of classes
P = {}

# Class1 routing: Queue1 -> Queue2 -> stays as Class1, Queue3 -> Queue1 as Class2
P[(jobclass1, jobclass1)] = np.array([[0,1,0], [0,0,1], [0,0,0]])
P[(jobclass1, jobclass2)] = np.array([[0,0,0], [0,0,0], [1,0,0]])
P[(jobclass1, jobclass3)] = np.array([[0,0,0], [0,0,0], [0,0,0]])
P[(jobclass1, jobclass4)] = np.array([[0,0,0], [0,0,0], [0,0,0]])

# Class2 routing: Queue3 -> Queue1 as Class1
P[(jobclass2, jobclass1)] = np.array([[0,0,0], [0,0,0], [1,0,0]])
P[(jobclass2, jobclass2)] = np.array([[0,1,0], [0,0,1], [0,0,0]])
P[(jobclass2, jobclass3)] = np.array([[0,0,0], [0,0,0], [0,0,0]])
P[(jobclass2, jobclass4)] = np.array([[0,0,0], [0,0,0], [0,0,0]])

# Class3 routing: Queue3 -> Queue1 as Class4
P[(jobclass3, jobclass1)] = np.array([[0,0,0], [0,0,0], [0,0,0]])
P[(jobclass3, jobclass2)] = np.array([[0,0,0], [0,0,0], [0,0,0]])
P[(jobclass3, jobclass3)] = np.array([[0,1,0], [0,0,1], [0,0,0]])
P[(jobclass3, jobclass4)] = np.array([[0,0,0], [0,0,0], [1,0,0]])

# Class4 routing: Queue3 -> Queue1 as Class3, Queue1 -> Queue3
P[(jobclass4, jobclass1)] = np.array([[0,0,0], [0,0,0], [0,0,0]])
P[(jobclass4, jobclass2)] = np.array([[0,0,0], [0,0,0], [0,0,0]])
P[(jobclass4, jobclass3)] = np.array([[0,0,0], [0,0,0], [1,0,0]])
P[(jobclass4, jobclass4)] = np.array([[0,0,1], [0,0,0], [0,0,0]])

model.link(P)

In [15]:
# Set initial state: all jobs at Queue3
n = np.array([[0,0,0,0],        # Queue1: no jobs
              [0,0,0,0],        # Queue2: no jobs
              [N[0],N[1],N[2],N[3]]])  # Queue3: all jobs

nodes = [node1, node2, node3]
for i in range(len(nodes)):
    nodes[i].set_state(n[i])

state = model.state()

In [16]:
# Solver options
options = Solver.defaultOptions
options.verbose = 1

# Results storage
results = {}
timings = {}

In [17]:
# CTMC solver
start_time = time.time()
options.seed = 23000
solver_ctmc = CTMC(model, options)
Pr_ctmc = solver_ctmc.prob_sys_aggr()

CTMC getProbSysAggr failed: java.lang.IllegalArgumentException: Outside of matrix bounds


In [18]:
# JMT solver (simulation)
start_time = time.time()
solver_jmt = JMT(model, seed=23000, samples=100000)
Pr_jmt = solver_jmt.prob_sys_aggr()

Jul 29, 2025 12:13:21 PM jline.io.InputOutputKt line_printf
SEVERE: [LINE2JSIMG] Error during LINE to JSIMG conversion: Unable to determine root folder.
Error in sampleSysAggr: Error during network copy operation: [LINE2JSIMG] Error during LINE to JSIMG conversion: Unable to determine root folder.


In [19]:
# NC solver with sampling
start_time = time.time()
options_nc = Solver.defaultOptions
options_nc.verbose = 1
options_nc.method = 'ls'  # Logistic sampling
options_nc.seed = 23000
options_nc.samples = 1000

solver_nc = NC(model, options_nc)
Pr_nc = solver_nc.prob_sys_aggr()